# 1.将所有流程用keras高层接口实现

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
#载入数据
(x_train,y_train),(x_val,y_val)=keras.datasets.fashion_mnist.load_data()
print('训练集维度：',x_train.shape,y_train.shape)
print('数据范围:',x_train.min(),x_train.max())
print('类别：',set(y_train))
print('类别数：',len(set(y_train)))
print('验证集维度：',x_val.shape,y_val.shape)

训练集维度： (60000, 28, 28) (60000,)
数据范围: 0 255
类别： {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
类别数： 10
验证集维度： (10000, 28, 28) (10000,)


In [3]:
#预处理数据
def preprocess(x,y):
    x = tf.cast(x,dtype=tf.float32)/255
    x = tf.reshape(x,[-1,784])
    y = tf.cast(y,dtype=tf.int32)
    y = tf.one_hot(y,depth=10)
    return x,y

In [4]:
#准备每次喂入网络的数据batch
#每个batch200组数据，用preprocess处理这些数据，随机打乱（打乱时内存分配空间为10000）
db_train = tf.data.Dataset.from_tensor_slices((x_train,y_train)).batch(200).map(preprocess).shuffle(10000)
db_val = tf.data.Dataset.from_tensor_slices((x_val,y_val)).batch(200).map(preprocess).shuffle(10000)

db_iter = iter(db_train)
sample = next(db_iter)
print('每次喂入的样本量:',sample[0].shape)
print('每次喂入的标签量:',sample[1].shape)

每次喂入的样本量: (200, 784)
每次喂入的标签量: (200, 10)


In [5]:
#定义网络
model= keras.Sequential([
    keras.layers.Dense(512,activation=tf.nn.relu),
    keras.layers.Dense(256,activation=tf.nn.relu),
    keras.layers.Dense(128,activation=tf.nn.relu),
    keras.layers.Dense(64,activation=tf.nn.relu),
    keras.layers.Dense(10,activation = tf.nn.softmax)
])

In [8]:
#配置网络
#用Adam进行优化，学习率为0.01，损失函数为交叉熵损失，评价准则为精度
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01),loss=tf.losses.categorical_crossentropy,metrics=['accuracy'])

In [9]:
#训练数据
#训练集为db_train,训练是个循环，验证集为db_val,每2步验证一次
model.fit(db_train,epochs=10,validation_data=db_val,validation_steps=2)

Epoch 1/10
300/300 [==============================] - 4s 12ms/step - loss: 0.6631 - accuracy: 0.7666 - val_loss: 0.3503 - val_accuracy: 0.8900
Epoch 2/10
300/300 [==============================] - 3s 8ms/step - loss: 0.4072 - accuracy: 0.8521 - val_loss: 0.3346 - val_accuracy: 0.8925
Epoch 3/10
300/300 [==============================] - 3s 9ms/step - loss: 0.3700 - accuracy: 0.8650 - val_loss: 0.2980 - val_accuracy: 0.8750
Epoch 4/10
300/300 [==============================] - 3s 9ms/step - loss: 0.3477 - accuracy: 0.8735 - val_loss: 0.2692 - val_accuracy: 0.9000
Epoch 5/10
300/300 [==============================] - 3s 9ms/step - loss: 0.3326 - accuracy: 0.8777 - val_loss: 0.2650 - val_accuracy: 0.8900
Epoch 6/10
300/300 [==============================] - 3s 9ms/step - loss: 0.3203 - accuracy: 0.8828 - val_loss: 0.2875 - val_accuracy: 0.8875
Epoch 7/10
300/300 [==============================] - 3s 9ms/step - loss: 0.3129 - accuracy: 0.8847 - val_loss: 0.2917 - val_accuracy: 0.8850
Epoch

In [10]:
#预测
model.evaluate(db_val)

50/50 [==============================] - 0s 5ms/step - loss: 0.4015 - accuracy: 0.8688


[0.4014942428469658, 0.8688]